# Wissensaggregator Mittelalter und frühe Neuzeit
WIAGweb2

## Daten für Domherren aus dem Digitalen Personenregister einlesen

Übertrage Daten aus der GS-online. Die Daten werden aus der produktiven Datenbank auf eine lokale Datenbank eingelesen: `gso`

[Amtsperiode](#Amtsperiode)  
[Zeitspanne der Person](#Zeitspanne-der-Person)  
[Sortierung der Person nach Amt](#Sortierung-der-Person-nach-Amt) (hinfällig?)

Vorspann: Pakete laden; mit der Datenbank verbinden

In [1]:
wds_path="../.."

"../.."

In [2]:
cd(wds_path)

In [3]:
using Pkg

In [4]:
Pkg.activate(".")

  Activating project at `C:\Users\georg\Documents\projekte\WiagDataSetup.jl`


In [5]:
using Revise, MySQL

In [6]:
using WiagDataSetup

In [7]:
using MySQL, DataFrames, CSV

In [8]:
Wds = WiagDataSetup

WiagDataSetup

In [9]:
Wds.setDBWIAG(user="georg", db="wiag2")

Passwort für User georg: ········


MySQL.Connection(host="127.0.0.1", user="georg", port="3306", db="wiag2")

In [10]:
item_type_id = 6

6

Quelle: *2022-02-25* `gso_in_202202`

In [11]:
gs_db = "gso_in_202202"

"gso_in_202202"

Die Ämter in Tabelle `role` werden im Notebook "Strukturdaten" eingeladen.

## Amtsperiode

In [12]:
table_name = gs_db * ".offices";
sql = "SELECT * FROM $(table_name)
WHERE deleted = 0";
df_pr = Wds.sql_df(sql);

In [13]:
Wds.clean_up!(df_pr);

In [14]:
size(df_pr)

(72637, 22)

### Domherren zuordnen

In [15]:
transform!(df_pr, :person_id => ByRow(string) => :id_in_source);

In [22]:
table_name = "item";
sql = "SELECT id AS item_id, id_in_source FROM $(table_name) WHERE item_type_id = $(item_type_id)";
df_idx = Wds.sql_df(sql);

In [24]:
size(unique(df_idx.item_id))

(8523,)

In [25]:
df_prp = innerjoin(df_pr, df_idx, on = :id_in_source);

In [26]:
size(df_prp), size(unique(df_prp.person_id))

((15028, 24), (8509,))

In [27]:
person_id_mg_oc = setdiff(df_idx.item_id, df_prp.item_id)

14-element Vector{Int32}:
 162402
 162403
 162405
 162406
 162415
 162442
 162443
 162445
 162446
 162451
 162453
 162456
 162466
 162487

In [28]:
str_mg = "(" * join(person_id_mg_oc, ", ") * ")"

"(162402, 162403, 162405, 162406, 162415, 162442, 162443, 162445, 162446, 162451, 162453, 162456, 162466, 162487)"

*2022-03-10* 14 Domherren sind ohne Amt. Das ist möglich, weil Domherren ergänzt werden, die referenziert werden von der Domherrendatenbank unabhängig von einem Amt

### Rollen/Ämter zuordnen

In [29]:
table_name = "role"
sql = "SELECT name as role_name, role.id as role_id
FROM $(table_name)"
df_role = Wds.sql_df(sql);

In [30]:
df_prp = leftjoin(df_prp, df_role, on = :bezeichnung => :role_name);

In [31]:
df_mg_role_id = subset(df_prp, :role_id => ByRow(ismissing));

In [32]:
size(df_prp, 1), size(df_mg_role_id, 1)

(15028, 1214)

*2022-01-20* Es gibt noch kein Mapping von Ämtern auf normierte Ämter. Die Suche der Webanwendung berücksichtigt aber auch die ursprünglichen Bezeichnungen (`person_role.role_name`).

### Bistümer zuordnen

In [33]:
sql = "SELECT id AS diocese_id, name AS diocese_name FROM diocese"
df_dioc = Wds.sql_df(sql);

In [34]:
df_prp = leftjoin(df_prp, df_dioc, on = :dioezese => :diocese_name, matchmissing = :notequal);

### Instutionen zuordnen
Klöster, Domstifte

`klosterid` entspricht `institution.id_gsn`

In [35]:
sql = "SELECT id AS institution_id, name as institution_name, id_gsn FROM institution"
df_inst = Wds.sql_df(sql);

In [36]:
names(df_inst)

3-element Vector{String}:
 "institution_id"
 "institution_name"
 "id_gsn"

In [37]:
df_prp[100:105, [:bezeichnung, :klosterid, :institution]]

,bezeichnung,klosterid,institution
,String?,String?,String?
1,Domherr,832,Domstift Hildesheim
2,Domherr,832,Domstift Hildesheim
3,Domherr,832,Domstift Hildesheim
4,Domherr,832,Domstift Hildesheim
5,Domherr,832,Domstift Hildesheim
6,Domherr,832,Domstift Hildesheim


In [38]:
int_or_missing(x) = ismissing(x) ? missing : parse(Int, x);
transform!(df_prp, :klosterid => ByRow(int_or_missing) => :klosterid);

In [39]:
df_prp = leftjoin(df_prp, df_inst, on = :klosterid => :id_gsn, matchmissing = :notequal);

In [40]:
df_prp[100:105, [:bezeichnung, :klosterid, :institution, :institution_id]]

,bezeichnung,klosterid,institution,institution_id
,String?,Int64?,String?,Int32?
1,Domherr,832,Domstift Hildesheim,80025
2,Domherr,2066,Domstift Schwerin,80300
3,Domherr,832,Domstift Hildesheim,80025
4,Domvikar,803,Domstift Osnabrück,79983
5,Domvikar,628,Domstift Münster,79825
6,Kanoniker,175,Stift St. Nikolaus,79532


### Numerische Daten eintragen

In [44]:
num_date_begin(s_date) = Wds.parsemaybe(s_date, :lower)
transform!(df_prp, :von => ByRow(num_date_begin) => :num_date_begin);

In [45]:
num_date_end(s_date) = Wds.parsemaybe(s_date, :upper);
transform!(df_prp, :bis => ByRow(num_date_end) => :num_date_end);

In [46]:
transform!(df_prp, :bis => ByRow(Wds.parse_year_sort) => :date_sort_key);

In [47]:
function update_col(from, to)
    cand = Wds.parse_year_sort(from)
    if ismissing(to) || cand < to
        return cand
    else
        return to
    end
end

update_col (generic function with 1 method)

In [48]:
transform!(df_prp, [:von, :date_sort_key] => ByRow(update_col) => :date_sort_key);

In [49]:
df_prp[560:567, [:person_id, :bezeichnung, :von, :num_date_begin, :bis, :num_date_end, :date_sort_key]]

,person_id,bezeichnung,von,num_date_begin,bis,num_date_end,date_sort_key
,Int32,String?,String?,Int64?,String?,Int64?,Int64
1,282797,Domherr,1360,1360,1367,1367,1360150
2,282797,Kantor,1367,1367,missing,missing,1367150
3,282802,Domherr,1707,1707,1721,1721,1707150
4,282802,Kanoniker,missing,missing,missing,missing,9000900
5,282802,Domherr,1703,1703,missing,missing,1703150
6,282802,Domherr,vor 1721,1671,missing,missing,1721100
7,282802,Domherr,vor 1721,1671,missing,missing,1721100
8,282820,Kanoniker,vor 1797,1747,missing,missing,1797100


Orte zuordnen. Dann sind Daten redundant in der Datenbank, das erleichtert aber die Suche.
*2022-02-28* Verfolge die Idee zunächst nicht weiter, sondern frage die Orte in der Anwendung ab.

Einträge für diesen Item_Typ löschen

In [50]:
item_type_id

6

In [51]:
table_name = "person_role"
sql = "DELETE FROM $(table_name) WHERE person_id IN 
(SELECT id FROM item WHERE item_type_id = $(item_type_id))"
DBInterface.execute(Wds.dbwiag, sql)

MySQL.TextCursor{true}(MySQL.Connection(host="127.0.0.1", user="georg", port="3306", db="wiag2"), "DELETE FROM person_role WHERE person_id IN \n(SELECT id FROM item WHERE item_type_id = 6)", 0, -1, 0, MySQL.API.MYSQL_RES(Ptr{Nothing} @0x0000000000000000), Symbol[], Type[], Dict{Symbol, Int64}(), 0, 1, false)

Daten schreiben

Schreibe für alle Bistümer - auch für die identifizierten - den Namen des Bistums in die Tabelle.  
Schreibe für alle Ämter - auch für die identifizieren - die Bezeichnung in die Tabelle.  
Schreibe für alle Institutionen - auch für die identifizieren - die Bezeichnung in die Tabelle.  

In [52]:
columns = [
    :dioezese => :diocese_name,
    :von => :date_begin,
    :bis => :date_end,
    :anmerkung => :note,
    :bezeichnung => :role_name,
    :item_id => :person_id,
    :diocese_id => :diocese_id,
    :role_id => :role_id,
    :institution_id => :institution_id,
    :institution_name => :institution_name,    
    :num_date_begin => :num_date_begin,
    :num_date_end => :num_date_end,
    :date_sort_key => :date_sort_key
];

In [53]:
table_name = "person_role"
Wds.filltable!(table_name, select(df_prp, columns))

┌ Info: 10000
└ @ WiagDataSetup C:\Users\georg\Documents\projekte\WiagDataSetup.jl\src\WiagDataSetup.jl:1185
┌ Info: Rows inserted: 15028
└ @ WiagDataSetup C:\Users\georg\Documents\projekte\WiagDataSetup.jl\src\WiagDataSetup.jl:1208


15028

## Zeitspanne der Person

In [45]:
table_name = "item"
sql = "SELECT id, num_date_birth, num_date_death
FROM person WHERE id in (SELECT id FROM $(table_name) WHERE item_type_id = $(item_type_id))"
df_date = Wds.sql_df(sql);

In [46]:
size(df_date)

(8523, 3)

Zeitspanne der Ämter

In [47]:
df_pr_begin_0 = dropmissing(df_prp, :num_date_begin)
df_pr_begin = groupby(select(df_pr_begin_0, :person_id, :num_date_begin), :person_id)

,person_id,num_date_begin
,Int32,Int64
1,153969,1410
,person_id,num_date_begin
,Int32,Int64
1,162491,1476
2,162491,1479


In [48]:
df_pr_date_min = combine(df_pr_begin, :num_date_begin => minimum => :date_min);

In [49]:
size(df_pr_date_min)

(8334, 2)

In [50]:
df_pr_date_min[3015:3021, :]

,person_id,date_min
,Int32,Int64
1,157106,1218
2,157107,1178
3,157108,1200
4,157109,1200
5,157110,1200
6,157111,1209
7,157112,987


`date_max`

In [51]:
df_pr_end_value = dropmissing(df_prp, :num_date_end)
df_pr_end = groupby(select(df_pr_end_value, :person_id, :num_date_end), :person_id)

,person_id,num_date_end
,Int32,Int64
1,153969,1411
,person_id,num_date_end
,Int32,Int64
1,162491,1513
2,162491,1513


In [52]:
df_pr_date_max = combine(df_pr_end, :num_date_end => maximum => :date_max);

In [53]:
size(df_pr_date_max)

(5402, 2)

In [54]:
df_pr_date_max[3015:3021, :]

,person_id,date_max
,Int32,Int64
1,159273,1435
2,159274,1350
3,159275,1476
4,159276,1538
5,159277,1428
6,159278,1495
7,159279,1803


In [55]:
df_date = leftjoin(df_date, df_pr_date_min, on = :id => :person_id);

In [56]:
df_date = leftjoin(df_date, df_pr_date_max, on = :id => :person_id);

Geburts- und Sterbedatum  
Geburts- und Sterbedaten - so vorhanden - können die Zeitspanne nur verkürzen!

In [57]:
min_may_be(x, y) = ismissing(x) ? y : (ismissing(y) ? x : min(x, y));
max_may_be(x, y) = ismissing(x) ? y : (ismissing(y) ? x : max(x, y));

In [58]:
transform!(df_date, [:date_min, :num_date_birth] => ByRow(max_may_be) => :date_min); # sic!

In [59]:
transform!(df_date, [:date_max, :num_date_death] => ByRow(min_may_be) => :date_max); # sic!

Übernimm' den jeweils gültigen Wert für fehlende Daten

In [60]:
get_first_not_missing(x, y) = !ismissing(x) ? x : y;

In [61]:
transform!(df_date, [:date_min, :date_max] => ByRow(get_first_not_missing) => :date_min);

In [62]:
transform!(df_date, [:date_max, :date_min] => ByRow(get_first_not_missing) => :date_max);

Damit sind Einträge mit fehlenden Daten entbehrlich

In [63]:
dropmissing!(df_date, :date_min);

Trage die geänderten Daten ein

In [65]:
sql = "DROP TABLE IF EXISTS date_update"
DBInterface.execute(Wds.dbwiag, sql)

MySQL.TextCursor{true}(MySQL.Connection(host="127.0.0.1", user="georg", port="3306", db="wiag2"), "DROP TABLE IF EXISTS date_update", 0, -1, 0, MySQL.API.MYSQL_RES(Ptr{Nothing} @0x0000000000000000), Symbol[], Type[], Dict{Symbol, Int64}(), 0, 1, false)

In [66]:
sql = "CREATE TEMPORARY TABLE date_update (id INT, date_min INT, date_max INT) 
ENGINE=InnoDB DEFAULT CHARSET=utf8;";
DBInterface.execute(Wds.dbwiag, sql)

MySQL.TextCursor{true}(MySQL.Connection(host="127.0.0.1", user="georg", port="3306", db="wiag2"), "CREATE TEMPORARY TABLE date_update (id INT, date_min INT, date_max INT) \nENGINE=InnoDB DEFAULT CHARSET=utf8;", 0, -1, 0, MySQL.API.MYSQL_RES(Ptr{Nothing} @0x0000000000000000), Symbol[], Type[], Dict{Symbol, Int64}(), 0, 1, false)

In [67]:
names(df_date)

5-element Vector{String}:
 "id"
 "num_date_birth"
 "num_date_death"
 "date_min"
 "date_max"

In [68]:
columns = [
    :id => :id,
    :date_min => :date_min,
    :date_max => :date_max
]

3-element Vector{Pair{Symbol, Symbol}}:
       :id => :id
 :date_min => :date_min
 :date_max => :date_max

In [69]:
df_date[1:10, :]

,id,num_date_birth,num_date_death,date_min,date_max
,Int32,Int32?,Int32?,Signed,Signed?
1,153969,missing,missing,1410,1411
2,153970,missing,missing,1543,1588
3,153980,missing,missing,1208,1209
4,153981,missing,missing,1233,1284
5,153982,missing,1341,1324,1341
6,153984,missing,missing,1469,1472
7,153988,missing,missing,1522,1535
8,153990,missing,1527,1464,1527
9,153991,missing,1397,1373,1397


*2022-02-20* nicht zu identifizierender Fehler: UndefRefError: access to undefined reference mit `Wds.filltable!`

Alternative, direkte Schleife

In [70]:
table_name = "date_update";
step_msg = 800;
for (i, row) in enumerate(eachrow(df_date))
    sql = "INSERT INTO $(table_name) " *
    "VALUES ($(row[:id]), $(row[:date_min]), $(row[:date_max]))"
    DBInterface.execute(Wds.dbwiag, sql)
    if i % step_msg == 0
        @info "Datensatz" i
    end
end

┌ Info: Datensatz
│   i = 800
└ @ Main In[70]:8
┌ Info: Datensatz
│   i = 1600
└ @ Main In[70]:8
┌ Info: Datensatz
│   i = 2400
└ @ Main In[70]:8
┌ Info: Datensatz
│   i = 3200
└ @ Main In[70]:8
┌ Info: Datensatz
│   i = 4000
└ @ Main In[70]:8
┌ Info: Datensatz
│   i = 4800
└ @ Main In[70]:8
┌ Info: Datensatz
│   i = 5600
└ @ Main In[70]:8
┌ Info: Datensatz
│   i = 6400
└ @ Main In[70]:8
┌ Info: Datensatz
│   i = 7200
└ @ Main In[70]:8
┌ Info: Datensatz
│   i = 8000
└ @ Main In[70]:8


In [71]:
sql = "UPDATE person AS p, 
(SELECT id, date_min, date_max FROM date_update) AS du
SET p.date_min = du.date_min, p.date_max = du.date_max
WHERE p.id = du.id";
DBInterface.execute(Wds.dbwiag, sql)

MySQL.TextCursor{true}(MySQL.Connection(host="127.0.0.1", user="georg", port="3306", db="wiag2"), "UPDATE person AS p, \n(SELECT id, date_min, date_max FROM date_update) AS du\nSET p.date_min = du.date_min, p.date_max = du.date_max\nWHERE p.id = du.id", 0, -1, 8355, MySQL.API.MYSQL_RES(Ptr{Nothing} @0x0000000000000000), Symbol[], Type[], Dict{Symbol, Int64}(), 0, 1, false)

## Sortierung der Person nach Amt
Die Tabelle `person_display_order` ist eine technische Hilfstabelle, um die Abfragen in Doctrine zu vereinfachen oder überhaupt erst zu ermöglichen. Jede Person erhält für jedes Bistum einen Sortierschlüssel und einen weiteren unabhängig vom Bistum.

*2022-01-17* Verzichte zunächst auf diese Tabelle und löse das Problem in der Web-Anwendung. So bleibt das Datenmodell schlanker.

In [45]:
using MySQL, DataFrames

Sortierung unabhängig von einer Diözese. Es werden auch Ämter berücksichtigt, die nicht mit einer Diözese verbunden sind.

In [48]:
sql = "SELECT person_id, 'any' as diocese, min(display_order) as display_order " *
"FROM person_role GROUP BY person_id";
df_do = db_exec(sql);

In [49]:
size(df_do)

(5159, 3)

Zwölf Bischöfe haben kein Amt!?

In [50]:
table_dst = "person_display_order"
Wds.filltable!(table_dst, df_do)

┌ Info: Rows inserted: 5159
└ @ WiagDataSetup C:\Users\Georg\Documents\projekte\WiagDataSetup.jl\src\WiagDataSetup.jl:1169


5159

Sortierung nach Diözese

In [52]:
sql = "SELECT person_id, diocese_name as diocese, min(display_order) as display_order " *
"FROM person_role " *
"WHERE diocese_name IS NOT NULL " *
"GROUP BY person_id, diocese_name";
df_do = db_exec(sql);

In [53]:
size(df_do)

(6370, 3)

In [54]:
table_dst = "person_display_order"
Wds.filltable!(table_dst, df_do)

┌ Info: Rows inserted: 6370
└ @ WiagDataSetup C:\Users\Georg\Documents\projekte\WiagDataSetup.jl\src\WiagDataSetup.jl:1169


6370

Bischöfe ohne Ämter

In [55]:
sql = "SELECT id AS person_id, 'any' as diocese, 950000 AS display_order " *
"FROM person WHERE id NOT IN (SELECT DISTINCT person_id FROM person_role)"
df_do_r = db_exec(sql);

In [56]:
size(df_do_r)

(12, 3)

In [57]:
table_dst = "person_display_order"
Wds.filltable!(table_dst, df_do_r)

┌ Info: Rows inserted: 12
└ @ WiagDataSetup C:\Users\Georg\Documents\projekte\WiagDataSetup.jl\src\WiagDataSetup.jl:1169


12